In [163]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.svm import SVC
from wordcloud import WordCloud
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectFromModel, SelectPercentile
from sklearn.metrics import f1_score, confusion_matrix, classification_report, accuracy_score,precision_score, recall_score,roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score

In [164]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [165]:
df_main = pd.read_csv('superstore_data.csv')
df_main

,Id,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Response,Complain
0,1826,1970,Graduation,Divorced,84835.0,0,0,6/16/2014,0,189,...,111,189,218,1,4,4,6,1,1,0
1,1,1961,Graduation,Single,57091.0,0,0,6/15/2014,0,464,...,7,0,37,1,7,3,7,5,1,0
2,10476,1958,Graduation,Married,67267.0,0,1,5/13/2014,0,134,...,15,2,30,1,3,2,5,2,0,0
3,1386,1967,Graduation,Together,32474.0,1,1,11/5/2014,0,10,...,0,0,0,1,1,0,2,7,0,0
4,5371,1989,Graduation,Single,21474.0,1,0,8/4/2014,0,6,...,11,0,34,2,3,1,2,7,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10142,1976,PhD,Divorced,66476.0,0,1,7/3/2013,99,372,...,47,48,78,2,5,2,11,4,0,0
2236,5263,1977,2n Cycle,Married,31056.0,1,0,1/22/2013,99,5,...,3,8,16,1,1,0,3,8,0,0
2237,22,1976,Graduation,Divorced,46310.0,1,0,3/12/2012,99,185,...,15,5,14,2,6,1,5,8,0,0
2238,528,1978,Graduation,Married,65819.0,0,0,11/29/2012,99,267,...,149,165,63,1,5,4,10,3,0,0


In [166]:
df_main.isnull().sum()

Id                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
Response                0
Complain                0
dtype: int64

In [167]:
df_main.dropna(inplace=True)

In [168]:
#frequency
df_main['OvrFrequency'] = df_main['NumWebPurchases'] + df_main['NumCatalogPurchases'] + df_main['NumStorePurchases'] 
#Avg order value
df_main['AvgOrderValue'] = (df_main['MntWines'] + df_main['MntFruits'] + df_main['MntMeatProducts'] + df_main['MntFishProducts'] + df_main['MntSweetProducts'] + df_main['MntGoldProds'])/df_main['NumWebPurchases'] + df_main['NumCatalogPurchases'] + df_main['NumStorePurchases'] + df_main['NumDealsPurchases']
#pct income spent
df_main['pct_income_spent'] = df_main['AvgOrderValue']/df_main['Income']
#deals per purchase
df_main['DealsPerPurchase'] = df_main['NumDealsPurchases']/df_main['OvrFrequency']

In [169]:
df_main[df_main['DealsPerPurchase'].isnull()].AvgOrderValue

456     inf
1291    inf
1826    inf
1925    inf
Name: AvgOrderValue, dtype: float64

In [170]:
#age as of enrollment
from datetime import datetime

df_main['precise_date'] = pd.to_datetime(df_main['Dt_Customer'])
df_main['age'] = df_main['precise_date'].dt.year - df_main['Year_Birth']

In [171]:
df_main.isnull().sum()

Id                     0
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
Response               0
Complain               0
OvrFrequency           0
AvgOrderValue          0
pct_income_spent       0
DealsPerPurchase       4
precise_date           0
age                    0
dtype: int64

In [172]:
df_main.dropna(inplace = True)

In [173]:
df_main.isnull().sum()

Id                     0
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
Response               0
Complain               0
OvrFrequency           0
AvgOrderValue          0
pct_income_spent       0
DealsPerPurchase       0
precise_date           0
age                    0
dtype: int64

In [174]:
##dropping unecessary vars
df_main.drop(['Id', 'Year_Birth','Dt_Customer','precise_date'], axis=1,inplace=True)

In [175]:
df_main.columns

Index(['Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome',
       'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts',
       'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
       'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
       'NumStorePurchases', 'NumWebVisitsMonth', 'Response', 'Complain',
       'OvrFrequency', 'AvgOrderValue', 'pct_income_spent', 'DealsPerPurchase',
       'age'],
      dtype='object')

In [176]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2212 entries, 0 to 2239
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Education            2212 non-null   object 
 1   Marital_Status       2212 non-null   object 
 2   Income               2212 non-null   float64
 3   Kidhome              2212 non-null   int64  
 4   Teenhome             2212 non-null   int64  
 5   Recency              2212 non-null   int64  
 6   MntWines             2212 non-null   int64  
 7   MntFruits            2212 non-null   int64  
 8   MntMeatProducts      2212 non-null   int64  
 9   MntFishProducts      2212 non-null   int64  
 10  MntSweetProducts     2212 non-null   int64  
 11  MntGoldProds         2212 non-null   int64  
 12  NumDealsPurchases    2212 non-null   int64  
 13  NumWebPurchases      2212 non-null   int64  
 14  NumCatalogPurchases  2212 non-null   int64  
 15  NumStorePurchases    2212 non-null   int64 

In [177]:
df_main[['Response']].value_counts()/(1883+333) #15% of class 1 implies imbalance

Response
0           0.847924
1           0.150271
Name: count, dtype: float64

In [178]:
# Select object columns
#object_columns = df_merged.select_dtypes(include=['object']).columns.tolist()
object_columns = ['Education','Marital_Status','Complain']
dummy_df = pd.get_dummies(df_main, columns=object_columns)

In [179]:
dummy_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2212 entries, 0 to 2239
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Income                   2212 non-null   float64
 1   Kidhome                  2212 non-null   int64  
 2   Teenhome                 2212 non-null   int64  
 3   Recency                  2212 non-null   int64  
 4   MntWines                 2212 non-null   int64  
 5   MntFruits                2212 non-null   int64  
 6   MntMeatProducts          2212 non-null   int64  
 7   MntFishProducts          2212 non-null   int64  
 8   MntSweetProducts         2212 non-null   int64  
 9   MntGoldProds             2212 non-null   int64  
 10  NumDealsPurchases        2212 non-null   int64  
 11  NumWebPurchases          2212 non-null   int64  
 12  NumCatalogPurchases      2212 non-null   int64  
 13  NumStorePurchases        2212 non-null   int64  
 14  NumWebVisitsMonth        2212

In [180]:
dummy_df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [181]:
dummy_df.isnull().sum()

Income                      0
Kidhome                     0
Teenhome                    0
Recency                     0
MntWines                    0
MntFruits                   0
MntMeatProducts             0
MntFishProducts             0
MntSweetProducts            0
MntGoldProds                0
NumDealsPurchases           0
NumWebPurchases             0
NumCatalogPurchases         0
NumStorePurchases           0
NumWebVisitsMonth           0
Response                    0
OvrFrequency                0
AvgOrderValue              44
pct_income_spent           44
DealsPerPurchase            2
age                         0
Education_2n Cycle          0
Education_Basic             0
Education_Graduation        0
Education_Master            0
Education_PhD               0
Marital_Status_Absurd       0
Marital_Status_Alone        0
Marital_Status_Divorced     0
Marital_Status_Married      0
Marital_Status_Single       0
Marital_Status_Together     0
Marital_Status_Widow        0
Marital_St

In [182]:
dummy_df.dropna(inplace=True)
dummy_df.isnull().sum()

Income                     0
Kidhome                    0
Teenhome                   0
Recency                    0
MntWines                   0
MntFruits                  0
MntMeatProducts            0
MntFishProducts            0
MntSweetProducts           0
MntGoldProds               0
NumDealsPurchases          0
NumWebPurchases            0
NumCatalogPurchases        0
NumStorePurchases          0
NumWebVisitsMonth          0
Response                   0
OvrFrequency               0
AvgOrderValue              0
pct_income_spent           0
DealsPerPurchase           0
age                        0
Education_2n Cycle         0
Education_Basic            0
Education_Graduation       0
Education_Master           0
Education_PhD              0
Marital_Status_Absurd      0
Marital_Status_Alone       0
Marital_Status_Divorced    0
Marital_Status_Married     0
Marital_Status_Single      0
Marital_Status_Together    0
Marital_Status_Widow       0
Marital_Status_YOLO        0
Complain_0    

In [183]:
df_new = dummy_df.fillna(0)

In [184]:
from sklearn.model_selection import train_test_split
X = dummy_df.drop('Response',axis = 1)
y = dummy_df['Response']

In [185]:
X

,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,Marital_Status_Absurd,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO,Complain_0,Complain_1
0,84835.0,0,0,0,189,104,379,111,189,218,...,False,False,True,False,False,False,False,False,True,False
1,57091.0,0,0,0,464,5,64,7,0,37,...,False,False,False,False,True,False,False,False,True,False
2,67267.0,0,1,0,134,11,59,15,2,30,...,False,False,False,True,False,False,False,False,True,False
3,32474.0,1,1,0,10,0,1,0,0,0,...,False,False,False,False,False,True,False,False,True,False
4,21474.0,1,0,0,6,16,24,11,0,34,...,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,66476.0,0,1,99,372,18,126,47,48,78,...,False,False,True,False,False,False,False,False,True,False
2236,31056.0,1,0,99,5,10,13,3,8,16,...,False,False,False,True,False,False,False,False,True,False
2237,46310.0,1,0,99,185,2,88,15,5,14,...,False,False,True,False,False,False,False,False,True,False
2238,65819.0,0,0,99,267,38,701,149,165,63,...,False,False,False,True,False,False,False,False,True,False


In [186]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Fit and transform the data
X_scaled = scaler.fit_transform(X)

X_scaled

array([[ 1.32226927, -0.81983225, -0.93280998, ..., -0.03038686,
         0.09649346, -0.09649346],
       [ 0.18921166, -0.81983225, -0.93280998, ..., -0.03038686,
         0.09649346, -0.09649346],
       [ 0.6047968 , -0.81983225,  0.91070144, ..., -0.03038686,
         0.09649346, -0.09649346],
       ...,
       [-0.25108152,  1.04718069, -0.93280998, ..., -0.03038686,
         0.09649346, -0.09649346],
       [ 0.54566087, -0.81983225, -0.93280998, ..., -0.03038686,
         0.09649346, -0.09649346],
       [ 1.73213685, -0.81983225,  2.75421287, ..., -0.03038686,
         0.09649346, -0.09649346]])

In [187]:
pip install --upgrade scikit-learn imbalanced-learn numpy

Note: you may need to restart the kernel to use updated packages.


In [188]:
conda install mkl

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
\ DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/1.1" 304 0
/ DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/osx-arm64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/osx-arm64/current_repodata.json HTTP/1.1" 304 0
done
Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): rep

In [189]:
from imblearn.over_sampling import RandomOverSampler

# Create a RandomOverSampler instance
ros = RandomOverSampler(sampling_strategy=0.4)

# Fit and resample the data
X_resampled, y_resampled = ros.fit_resample(X_scaled, y)

In [190]:
y_resampled.value_counts()/(1836+550)

Response
0    0.769489
1    0.307628
Name: count, dtype: float64

In [191]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled, random_state=104,  test_size=0.25,  shuffle=True)

## Neural Network

In [159]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np


train_accuracies = []
test_accuracies = []

# Initialize the neural network classifier
# Using a single hidden layer with 3 neurons
# The output layer will have 2 neurons, suitable for binary classification
mlp = MLPClassifier(hidden_layer_sizes=(3,), max_iter=1, warm_start=True, solver='sgd')

epoch_num = 500
# Train for 50 epochs
for epoch in range(epoch_num):
    # Fit model on training data
    mlp.fit(X_train, y_train)

    # Calculate and store training accuracy
    train_pred = mlp.predict(X_train)
    train_accuracy = accuracy_score(y_train, train_pred)
    train_accuracies.append(train_accuracy)

    # Calculate and store test accuracy
    test_pred = mlp.predict(X_test)
    test_accuracy = accuracy_score(y_test, test_pred)
    test_accuracies.append(test_accuracy)

# Display final training and test accuracies
final_train_accuracy = train_accuracies[-1]
final_test_accuracy = test_accuracies[-1]
final_train_accuracy, final_test_accuracy

/Users/zoyafaiz/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/zoyafaiz/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/zoyafaiz/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/zoyafaiz/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/zoyafaiz/anaconda3/lib/py

(0.8053969901401141, 0.8118195956454122)

print(classification_report(y_test, y_pred))

In [204]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.99      0.86       475
           1       0.82      0.08      0.15       168

    accuracy                           0.76       643
   macro avg       0.79      0.54      0.50       643
weighted avg       0.77      0.76      0.67       643

